In [1]:
# Ajustando locale da JVM para en-US, estava misturando os locale e ficando en-BR...
# Fonte: https://stackoverflow.com/questions/55246080/pyspark-stopwordsremover-parameter-locale-given-invalid-value
locale = spark.sparkContext._jvm.java.util.Locale
locale.setDefault(locale.forLanguageTag("en-US"))

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1601104146332_0002,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
import os
from timeit import default_timer as timer
from datetime import datetime

import numpy as np

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover

def load_file(filePath, type, delimiter = None):
    if type == 'text':
        df = spark.read.load(filePath, format='text')
    else:
        df = spark.read.csv(filePath, inferSchema=True, header=True, sep=delimiter)
    return df

def concat_all_reviews_v1(reviews):
    """
        Concatena todas as reviews (linhas no DF) em um RDD no formato (1, TEXTO).
        Retorna um RDD
    """
    rdd = reviews.rdd.map(lambda r: (1, r[0])).reduceByKey(lambda a, b: a + ' ' + b)
    return rdd

def concat_all_reviews_v2(reviews):
    """
        Concatena todas as reviews (linhas no DF) em um texto único.
        Retorna uma string
    """
    rdd = reviews.rdd.flatMap(lambda v: v[0].split()).reduce(lambda a, b: a + ' ' + b)
    return rdd

def tokenize_reviews(reviews):
    tokenization = Tokenizer(inputCol='review', outputCol='review_words')
    return tokenization.transform(reviews)

def remove_stop_words_reviews(reviews):
    stopword_removal = StopWordsRemover(inputCol='review_words',outputCol='review_words_refined')
    return stopword_removal.transform(reviews)

def counter_words_rdd_flatMap(df, index):
    """
        Filtra e conta palavras usando RDD flatMap
    """
    filtered = refined_reviews \
                .rdd \
                .flatMap(lambda v: [(w, 1) for w in v[index] if w != '' and len(w) > 3]) \
                .reduceByKey(lambda c1, c2: c1 + c2)
    return filtered


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:

#hdfs_folder = 'hdfs://ip-172-31-91-35.ec2.internal:8020/user/livy/'

now = datetime.now()
execs_per_file = 10

files = [
    "mobile_test.tsv",
   # "mobile_test2x.tsv",
   # "mobile_test3x.tsv",
   # "mobile_test4x.tsv",
   # "mobile_test8x.tsv",
   # "mobile_test10k.tsv",
  #  "mobile_test100k.tsv",
  #  "mobile_test200k.tsv"
]

for f in files:    
    lines = []
    for r in np.arange(0, execs_per_file):
        print("Round " + str(r))
        
        print(f"Loading file: {f}")
        content = load_file(f"{f}", type='tsv', delimiter='\t')
        
        start = timer()
        print("Removing empty reviews...")
        content = content.select('review_body').filter(content.review_body != '')
        
        print("Concating content...")
        all_reviews_rdd = concat_all_reviews_v1(content)
        
        print("Creating DF to Mlib...")
        # Cria DataFrame para uso com a Mlib
        all_reviews_df = spark.createDataFrame(data=all_reviews_rdd, schema=['id', 'review'])
        
        print("Tokenization...")
        all_reviews_df_tokenized = tokenize_reviews(all_reviews_df)
        
        print("Removing StopWords...")
        refined_reviews = remove_stop_words_reviews(all_reviews_df_tokenized)
        
        print("Filter and counter Words...")
        filtered = counter_words_rdd_flatMap(refined_reviews, 3)
        
        end = timer()
        exec_time = end - start
        l = (f, str(r), "{:.4f}".format(exec_time))
        lines.append(l)
        print(f"Execution time:{exec_time} sec")
        print("---------------------------")
        
    print("Writing exec logs...")
    logs_execs = spark.createDataFrame(data=lines, schema=['file', 'round', 'time'])
    
    log_file = f"s3://aws-emr-resources-188023916529-us-east-1/notebooks/e-8IEIV3G0OF4JFD9X5P7TBISY6/execs_new6"
    logs_execs.repartition(1).write.csv(path=log_file, mode="append", header="true")
    
    # logs_execs.coalesce(1).write.format("text").option("header", "false").mode("append").save(log_file)

filtered.take(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Round 0
Loading file: mobile_test.tsv
Removing empty reviews...
Concating content...
Creating DF to Mlib...
Tokenization...
Removing StopWords...
Filter and counter Words...
Execution time:5.099673669999902 sec
---------------------------
Round 1
Loading file: mobile_test.tsv
Removing empty reviews...
Concating content...
Creating DF to Mlib...
Tokenization...
Removing StopWords...
Filter and counter Words...
Execution time:0.7620989409988397 sec
---------------------------
Round 2
Loading file: mobile_test.tsv
Removing empty reviews...
Concating content...
Creating DF to Mlib...
Tokenization...
Removing StopWords...
Filter and counter Words...
Execution time:0.6737163289981254 sec
---------------------------
Round 3
Loading file: mobile_test.tsv
Removing empty reviews...
Concating content...
Creating DF to Mlib...
Tokenization...
Removing StopWords...
Filter and counter Words...
Execution time:0.667520966999291 sec
---------------------------
Round 4
Loading file: mobile_test.tsv
Remo

In [ ]:
#spark.stop()